# Ripser C++ code Notebook Tutorial

This notebook is a survey tutorial for the C++ Ripser code module by Ulrich Bauer. This tutorial is a work in progress created by Alvaro Torras Casas, Cardiff University, 2022 for didactic and educational purposes. This is aimed at non-`C++` experts and non-TDA experts. If you are an expert, you might find this boring.

In this notebook, we consider the source code from the famous C++ Ripser module which computes persistent homology. The original code can be found on this repository: 

[1] Ulrich Bauer, https://github.com/Ripser/ripser, 2015–2021 

Also, the ideas of this code are very well explained at the following article:

[2] Bauer, U. Ripser: efficient computation of Vietoris–Rips persistence barcodes. J Appl. and Comput. Topology 5, 391–423 (2021). https://doi.org/10.1007/s41468-021-00071-5

This tutorial has no claims of originality, rather than for didactic and educational purposes. 
It is rather a notebook to go through some of the parts of the original code, breaking down it into small-easy-to-understand pieces. We also put some references to [1] and [2] along this text.

## Notebook setup:

Notice that for running this notebook you need to have a `c++` kernel installed in jupyterlab. This can be done thanks to `xeus-cling` (https://github.com/jupyter-xeus/xeus-cling) for which you will probably need to install before `miniconda`. Notice that even though `xeus-cling` maintainers say that they do not support packages for the Windows platform, you can still get around this problem by using Windows Subsystem for Linux. To know whether you have successfully installed the `c++` kernel, type:

`jupyter kernelspec list`

If `xcpp11`, `xcpp14` and `xcpp17` do not appear on the list, you might have to got to the folders where the `xeus-cling` kernels where installed and run

`jupyter kernelspec install xcpp11 xcpp14 xcpp17`

It took me a while to get this working though. 


## Ripser Tutorial:

First of all, to avoid any licence problems, we copy the associated licence below. Also, we omit some parts of the original code and might have modified it at some places. Also, the order in which we present the code here does not necessarily need to follow the original. However, the idea is that once one understands this tutorial, it should be easy to read the original code from [2].

In [1]:
/*
 Ripser: a lean C++ code for computation of Vietoris-Rips persistence barcodes
 MIT License
 Copyright (c) 2015–2021 Ulrich Bauer
 Permission is hereby granted, free of charge, to any person obtaining a copy
 of this software and associated documentation files (the "Software"), to deal
 in the Software without restriction, including without limitation the rights
 to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 copies of the Software, and to permit persons to whom the Software is
 furnished to do so, subject to the following conditions:
 The above copyright notice and this permission notice shall be included in all
 copies or substantial portions of the Software.
 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
 SOFTWARE.
 You are under no obligation whatsoever to provide any bug fixes, patches, or
 upgrades to the features, functionality or performance of the source code
 ("Enhancements") to anyone; however, if you choose to make your Enhancements
 available either publicly, or directly to the author of this software, without
 imposing a separate written license agreement for such Enhancements, then you
 hereby grant the following license: a non-exclusive, royalty-free perpetual
 license to install, use, modify, prepare derivative works, incorporate into
 other computer software, distribute, and sublicense such enhancements or
 derivative works thereof, in binary and source code form.
*/

Next, there are some imports of usual `C++` modules

In [1]:
#include <algorithm>
#include <cassert>
#include <chrono>
#include <cmath>
#include <fstream>
#include <iostream>
#include <numeric>
#include <queue>
#include <sstream>
#include <unordered_map>

Then, types for values, indices and coefficients are defined.

In [2]:
typedef float value_t;
typedef int64_t index_t;
typedef uint16_t coefficient_t;

Also, define number of coefficients and maximum simplex index

In [3]:
static const size_t num_coefficient_bits = 8;
static const index_t max_simplex_index = (index_t(1) << (8 * sizeof(index_t) - 1 - num_coefficient_bits)) - 1;

To avoid overflow of `index_t` variables, an overflow check is also created. We will be using coefficients through the notebook. This is indicated in [1] with a definition `USE_COEFFICIENTS`

In [4]:
void check_overflow(index_t i) {
    if (i > max_simplex_index)
        throw std::overflow_error(
            "simplex index " + std::to_string((uint64_t)i) +
            " in filtration is larger than maximum index " +
            std::to_string(max_simplex_index)
        );
}

The class below introduces binomial coefficient tables as objects. Here, recall the data object `std::vector` and how it is constructed. For example, for creating a vector with `6` entries of zeros we proceed as follows: 

In [5]:
std::vector<index_t> A_vect(6,0);
std::cout << "Size of A_vect: " << A_vect.size() << std::endl;
int idx = 2;
std::cout << "Value of A_vect[" << idx << "]: " << A_vect[idx] << std::endl;

Size of A_vect: 6
Value of A_vect[2]: 0


### Binomial Tables

Next, a class is defined such that it stores binomial coefficients. Some notes:
- First, notice that binomial coefficients are stored in a (k+1) x (n+1) matrix.
- It is checked whether the binomial coefficients overflow. However, here the right shift bitwise operation `>>` is used. Unfortunately, it is not totally clear to me why this is the case.

In [6]:
class binomial_coeff_table {
    std::vector<std::vector<index_t>> B;
    
    public:
        binomial_coeff_table(index_t n, index_t k) : B(k + 1, std::vector<index_t>(n + 1, 0)) {
            for (index_t i = 0; i <= n; ++i) {
                B[0][i] = 1;
                for (index_t j = 1; j < std::min(i, k + 1); ++j)
                    B[j][i] = B[j - 1][i - 1] + B[j][i - 1];
                if (i <= k) B[i][i] = 1;
                check_overflow(B[std::min(i >> 1, k)][i]);
            }
        }

        index_t operator()(index_t n, index_t k) const {
            assert(n < B.size() && k < B[n].size() && n >= k - 1);
            return B[k][n];
        }
};

We might create an istance of the class to read directly some binomial coefficients.

In [7]:
int n = 7;
int k = 4;
binomial_coeff_table bct_obj = binomial_coeff_table(n,k);

In [8]:
bct_obj(3,2)

3

In [9]:
// Running bct_obj(n+1, k+1) will stop the kernel (it breaks an assertion error)

Next, we print all values of $\binom{t}{r}$ for all $0 \leq t \leq n$ and all $0 \leq r \leq \textrm{min}(t,k)$.

In [10]:
for (int i=0; i <= n; i++) {
    for (int j=0; j <= std::min(i, k); j++) {
        std::cout << bct_obj(i,j) << ", ";
    }
    std::cout << std::endl;
}

1, 
1, 1, 
1, 2, 1, 
1, 3, 3, 1, 
1, 4, 6, 4, 1, 
1, 5, 10, 10, 5, 
1, 6, 15, 20, 15, 
1, 7, 21, 35, 35, 


### Primes and Inverses

The following checks if a coefficient is prime:
 - First, we check whether a number is even (using the bitwise operation `&`) or it is smaller than $2$. If so, it checks that it is 2.
 - Then, for all odd numbers, starting from $p=3$ and up to $p$ such that $p^2 <= n$, we check whether $p$ divides $n$. If this happens at some $p$ we return `False`.

In [11]:
bool is_prime(const coefficient_t n) {
    if (!(n & 1) || n < 2) return n == 2;
    for (coefficient_t p = 3; p <= n / p; p += 2)
        if (!(n % p)) return false;
    return true;
}

In [12]:
std::cout << std::boolalpha; //This boolalpha makes sure that booleans are displayed as strings
std::cout << "5 is prime: " << bool(is_prime(5)) << std::endl;
std::cout << "-1 is prime: " << is_prime(-1) << std::endl;
std::cout << "2 is prime: " << is_prime(2) << std::endl;

5 is prime: true
-1 is prime: false
2 is prime: true


Next, as indicated by the name, a vector of lenght `m` (a prime number) is created which already contains the inverses modulo `m` for all integrs in `[0,m-1]`. This supposedly should speedup computations as the inverses do not need to be computed on the go.

In [13]:
std::vector<coefficient_t> multiplicative_inverse_vector(const coefficient_t m) {
	std::vector<coefficient_t> inverse(m);
	inverse[1] = 1;
	// m = a * (m / a) + m % a
	// Multipying with inverse(a) * inverse(m % a):
	// 0 = inverse(m % a) * (m / a) + inverse(a)  (mod m)
	for (coefficient_t a = 2; a < m; ++a) inverse[a] = m - (inverse[m % a] * (m / a)) % m;
	return inverse;
}

**Example**

As an example, take `m=7`.

In [14]:
int len = 7;
std::vector<coefficient_t> inv_vect = multiplicative_inverse_vector(len);
std::cout << "inv_vect = (";
for (int i=0; i < len; i++) {
    std::cout << inv_vect[i] << " ,";
}
std::cout << ")" << std::endl;

inv_vect = (0 ,1 ,4 ,5 ,2 ,3 ,6 ,)


### Entries (`entry_t`)


Create an `entry_t` data structure for storing indices and coefficients. It is checked that it has the same size as an `index_t` variable.

In [15]:
struct entry_t {
	index_t index : 8 * sizeof(index_t) - num_coefficient_bits;
	coefficient_t coefficient : num_coefficient_bits;
	entry_t(index_t _index, coefficient_t _coefficient)
	    : index(_index), coefficient(_coefficient) {}
	entry_t(index_t _index) : index(_index), coefficient(0) {}
	entry_t() : index(0), coefficient(0) {}
};

static_assert(sizeof(entry_t) == sizeof(index_t), "size of entry_t is not the same as index_t");

Define several functions for entry handling. We must execute them in different cells for scope reasons.

In [16]:
entry_t make_entry(index_t i, coefficient_t c) { return entry_t(i, c); }

In [17]:
index_t get_index(const entry_t& e) { return e.index; }

In [18]:
index_t get_coefficient(const entry_t& e) { return e.coefficient; }

In [19]:
void set_coefficient(entry_t& e, const coefficient_t c) { e.coefficient = c; }

In [20]:
const entry_t& get_entry(const entry_t& e) { return e; }

Define a way of printing `entry_t`. Here we use a solution to a `xeus-cling` bug from here: https://github.com/root-project/cling/issues/184

In [35]:
#define INSERTION_OPERATOR operator<< // Workaround for a bug of cling

std::ostream& INSERTION_OPERATOR( std::ostream& stream, const entry_t& e) {
    stream << get_index(e) << ":" << get_coefficient(e);
    return stream;
};

#undef INSERTION_OPERATOR

**Example:**

In [22]:
entry_t x = make_entry(index_t(17), coefficient_t(8));

In [23]:
get_index(x)

17

In [24]:
get_coefficient(x)

8

In [25]:
set_coefficient(x,3);
get_coefficient(x)

3

In [38]:
std::cout << "Created entry " << x << std::endl;

Created entry 17:3


### Diameter-Idex and Index-Diameter pairs

Next, two pairs are defined: `diameter_index_t` and `index_diameter_t`. Which store a diameter (`float`) and an index.

In [39]:
typedef std::pair<value_t, index_t> diameter_index_t;

In [40]:
value_t get_diameter(const diameter_index_t& i) { return i.first; }

In [41]:
index_t get_index(const diameter_index_t& i) { return i.second; }

In [42]:
typedef std::pair<index_t, value_t> index_diameter_t;

In [43]:
index_t get_index(const index_diameter_t& i) { return i.first; }

In [44]:
value_t get_diameter(const index_diameter_t& i) { return i.second; }

A structure is created for diameters and entries

In [45]:
struct diameter_entry_t : std::pair<value_t, entry_t> {
	using std::pair<value_t, entry_t>::pair;
	diameter_entry_t(value_t _diameter, index_t _index, coefficient_t _coefficient)
	    : diameter_entry_t(_diameter, make_entry(_index, _coefficient)) {}
	diameter_entry_t(const diameter_index_t& _diameter_index, coefficient_t _coefficient)
	    : diameter_entry_t(get_diameter(_diameter_index),
	                       make_entry(get_index(_diameter_index), _coefficient)) {}
	diameter_entry_t(const diameter_index_t& _diameter_index)
	    : diameter_entry_t(get_diameter(_diameter_index),
	                       make_entry(get_index(_diameter_index), 0)) {}
	diameter_entry_t(const index_t& _index) : diameter_entry_t(0, _index, 0) {}
};

In [46]:
const entry_t& get_entry(const diameter_entry_t& p) { return p.second; }

In [47]:
entry_t& get_entry(diameter_entry_t& p) { return p.second; }

In [48]:
const index_t get_index(const diameter_entry_t& p) { return get_index(get_entry(p)); }

In [49]:
const coefficient_t get_coefficient(const diameter_entry_t& p) {
	return get_coefficient(get_entry(p));
}

In [50]:
const value_t& get_diameter(const diameter_entry_t& p) { return p.first; }

In [53]:
void set_coefficient(diameter_entry_t& p, const coefficient_t c) {
	set_coefficient(get_entry(p), c);
}

**Example**

In [54]:
diameter_entry_t diam_entry = diameter_entry_t(11.5, 3, 4);
std::cout << "Created diameter - entry pair:" << std::endl;
std::cout << "Diameter: " << get_diameter(diam_entry) << std::endl;
std::cout << "Entry: " << get_entry(diam_entry) << std::endl;
std::cout << "Index: " << get_index(diam_entry) << std::endl;
std::cout << "Coefficient: " << get_coefficient(diam_entry) << std::endl;

Created diameter - entry pair:
Diameter: 11.5
Entry: 3:4
Index: 3
Coefficient: 4


In [56]:
set_coefficient(diam_entry, 7);
std::cout << "Entry: " << get_entry(diam_entry) << std::endl;
std::cout << "Coefficient: " << get_coefficient(diam_entry) << std::endl;

Entry: 3:7
Coefficient: 7


### Comparing Diameters and Entries

In [ ]:
template <typename Entry> bool greater_diameter_or_smaller_index(const Entry& a, const Entry& b) {
	return (get_diameter(a) > get_diameter(b)) ||
	       ((get_diameter(a) == get_diameter(b)) && (get_index(a) < get_index(b)));
}

**Example**

In [62]:
diameter_entry_t diam_entry_1 = diameter_entry_t(11.5, 3, 1);
diameter_entry_t diam_entry_2 = diameter_entry_t(12, 7, 1);
diameter_entry_t diam_entry_3 = diameter_entry_t(12, 8, 1);

In [66]:
std::cout << greater_diameter_or_smaller_index(diam_entry_1, diam_entry_2) << std::endl;
std::cout << greater_diameter_or_smaller_index(diam_entry_2, diam_entry_3) << std::endl;

false
true


### Compressed Distance Matrix

In [68]:
enum compressed_matrix_layout { LOWER_TRIANGULAR, UPPER_TRIANGULAR };

In [82]:
template <compressed_matrix_layout Layout> struct compressed_distance_matrix {
	std::vector<value_t> distances;
	std::vector<value_t*> rows;

	compressed_distance_matrix(std::vector<value_t>&& _distances)
	    : distances(std::move(_distances)), rows((1 + std::sqrt(1 + 8 * distances.size())) / 2) {
		assert(distances.size() == size() * (size() - 1) / 2);
		init_rows();
	}

	template <typename DistanceMatrix>
	compressed_distance_matrix(const DistanceMatrix& mat)
	    : distances(mat.size() * (mat.size() - 1) / 2), rows(mat.size()) {
		init_rows();

		for (size_t i = 1; i < size(); ++i)
			for (size_t j = 0; j < i; ++j) rows[i][j] = mat(i, j);
	}

	value_t operator()(const index_t i, const index_t j) const;
	size_t size() const { return rows.size(); }
	void init_rows();
};

In [83]:
typedef compressed_distance_matrix<LOWER_TRIANGULAR> compressed_lower_distance_matrix;
typedef compressed_distance_matrix<UPPER_TRIANGULAR> compressed_upper_distance_matrix;

In [84]:
template <> void compressed_lower_distance_matrix::init_rows() {
	value_t* pointer = &distances[0];
	for (size_t i = 1; i < size(); ++i) {
		rows[i] = pointer;
		pointer += i;
	}
}

In [85]:
template <> void compressed_upper_distance_matrix::init_rows() {
	value_t* pointer = &distances[0] - 1;
	for (size_t i = 0; i < size() - 1; ++i) {
		rows[i] = pointer;
		pointer += size() - i - 2;
	}
}

In [86]:
template <>
value_t compressed_lower_distance_matrix::operator()(const index_t i, const index_t j) const {
	return i == j ? 0 : i < j ? rows[j][i] : rows[i][j];
}

In [87]:
template <>
value_t compressed_upper_distance_matrix::operator()(const index_t i, const index_t j) const {
	return i == j ? 0 : i > j ? rows[j][i] : rows[i][j];
}

Then, some hash maps and tables are created (there is an option for Robin Hood hashing which here we omit.)

In [48]:
template <class Key, class T, class H, class E> using hash_map = std::unordered_map<Key, T, H, E>;
template <class Key> using hash = std::hash<Key>;

In [49]:
check_overflow(-1)